<a href="https://colab.research.google.com/github/srujith7/AD-075/blob/main/12_feb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data=pd.read_csv("/content/fake_news.csv")
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
data=data.drop(['id'],axis=1)

In [5]:
data=data.fillna('')

In [6]:
data['content']=data['author']+' '+data['title']+' '+data['text']

In [7]:
data=data.drop(['author','title','text'],axis=1)

In [8]:
data['content']=data['content'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [9]:
data['content']=data['content'].str.replace('[^\w\s]','')

In [11]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import metrics

In [14]:
x=data[['content']]
y=data['label']

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=45,stratify=y)

In [16]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(14560, 1)
(6240, 1)
(14560,)
(6240,)


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()

In [18]:
tfidf_vect=TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}',max_features=5000)
tfidf_vect.fit(data['content'])
x_train_tfidf=tfidf_vect.transform(x_train['content'])
x_test_tfidf=tfidf_vect.transform(x_test['content'])

In [19]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import metrics
pclf=PassiveAggressiveClassifier()
pclf.fit(x_train_tfidf,y_train)
predictions=pclf.predict(x_test_tfidf)
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.96      0.97      0.97      3116
           1       0.97      0.96      0.97      3124

    accuracy                           0.97      6240
   macro avg       0.97      0.97      0.97      6240
weighted avg       0.97      0.97      0.97      6240



In [20]:
print(metrics.confusion_matrix(y_test,predictions))

[[3014  102]
 [ 110 3014]]


In [22]:
# Corrected import statement
from sklearn.neural_network import MLPClassifier

mlpclf = MLPClassifier(hidden_layer_sizes=(256, 64,16),activation = 'relu', solver = 'adam')
mlpclf.fit(x_train_tfidf,y_train)
predictions=mlpclf.predict(x_test_tfidf)
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96      3116
           1       0.97      0.96      0.96      3124

    accuracy                           0.96      6240
   macro avg       0.96      0.96      0.96      6240
weighted avg       0.96      0.96      0.96      6240



In [23]:
print(metrics.confusion_matrix(y_test,predictions))

[[3014  102]
 [ 125 2999]]


In [24]:
import pickle
pickle.dump(mlpclf, open('model.pkl', 'wb'))